In [1]:
import os
import requests
from pyspark.sql import SparkSession
from tmlt.analytics.privacy_budget import PureDPBudget
from tmlt.analytics.query_builder import QueryBuilder
from tmlt.analytics.session import Session

r = requests.get(
    'https://tumult-public.s3.amazonaws.com/library-members.csv',
)
with open("members.csv", "w") as f:
    f.write(r.text)
spark = SparkSession.builder.getOrCreate()
members_df = spark.read.csv("members.csv", header=True, inferSchema=True)

22/05/26 12:31:28 WARN Utils: Your hostname, anshu resolves to a loopback address: 127.0.1.1; using 192.168.1.56 instead (on interface wlp9s0)
22/05/26 12:31:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/26 12:31:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 12:31:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/26 12:31:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
session = Session.from_dataframe(
    privacy_budget=PureDPBudget(epsilon=float('inf')),
    source_id="members",
    dataframe=members_df,
)

In [7]:
mean_age_query = QueryBuilder("members").average("age", low=0, high=200)
mean_age = session.evaluate(
    mean_age_query,
    privacy_budget=PureDPBudget(epsilon=10)
)
mean_age.show()

+------------------+
|       age_average|
+------------------+
|38.045852776804324|
+------------------+



In [24]:
itms_list = members_df.select('id').collect()

In [25]:
itms_array = [int(row.id) for row in itms_list]

In [26]:
from tmlt.analytics.keyset import KeySet

edu_levels = KeySet.from_dict({
    "id": itms_array
})

In [27]:
len(itms_array)

54217

In [29]:
edu_average_age_query = (
    QueryBuilder("members")
    .groupby(edu_levels)
    .average("age", low=0, high=120)
)
edu_average_ages = session.evaluate(
    edu_average_age_query,
    privacy_budget=PureDPBudget(100),
)
edu_average_ages.sort("age_average").show(truncate=False)

KeyboardInterrupt: 